In [2]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime
from io import StringIO

# ---------- CONFIG ----------
SAVE_PATH = "data/raw/gold_history.csv"
BASE_URL = "https://xn--42cah7d0cxcvbbb9x.com/"
YEARS = [2566, 2567, 2568]
# ----------------------------
thai_months = [
    "มกราคม", "กุมภาพันธ์", "มีนาคม", "เมษายน", "พฤษภาคม", "มิถุนายน",
    "กรกฎาคม", "สิงหาคม", "กันยายน", "ตุลาคม", "พฤศจิกายน", "ธันวาคม"
]
# ----------------------------

# ---------- SETUP CHROME ----------
options = Options()
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)

# ---------- SCRAPE ----------
all_data = []

def fetch_table(url):
    """โหลดเว็บและแปลงตารางราคาทองเป็น DataFrame โดยจัดการกับ Header 2 ชั้น"""
    driver.get(url)
    time.sleep(5)  # รอให้ JS โหลดข้อมูลเสร็จ
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find("table")
    
    if not table:
        print(f"❌ ไม่พบแท็ก <table> ใน {url}")
        return None

    # 🌟 ปรับปรุง: กำหนด Headers ที่คาดหวังตามโครงสร้างของตาราง
    # โครงสร้างที่ถูกต้องคือ:
    # [วันที่/เวลา, ครั้งที่, ทองแท่งรับซื้อ, ทองแท่งขายออก, ทองรูปพรรณฐานภาษี, ทองรูปพรรณขายออก, Gold spot, เงินบาท, ขึ้น/ลง]
    expected_headers = [
        'date_time', 
        'round', 
        'gold_buy', 
        'gold_sell', 
        'gold_bar_buy', 
        'gold_bar_sell', 
        'gold_spot', 
        'thb_rate', 
        'diff'
    ]
    
    rows_data = []
    # ข้าม <tr> สองแถวแรกซึ่งเป็น Header (<thead>)
    data_rows = table.find("tbody").find_all("tr")
    
    for tr in data_rows:
        cells = [td.get_text(strip=True) for td in tr.find_all("td")]
        
        # กรองแถวที่ไม่ใช่ข้อมูลจริง (เช่น แถวสรุปรายวัน)
        # แถวข้อมูลจริงควรมี 9 คอลัมน์
        if len(cells) == len(expected_headers): 
            rows_data.append(cells)
        # ⚠️ แถวที่มีปัญหามักจะมีจำนวนคอลัมน์ไม่ตรงหรือไม่ใช่ข้อมูลราคา
        # เช่น แถวที่แสดง 'พุธที่ 25 มกราคม 66-100'

    if not rows_data:
        print(f"⚠️ ไม่มีข้อมูลราคาที่สมบูรณ์ในตาราง: {url}")
        return None
        
    df = pd.DataFrame(rows_data, columns=expected_headers)
    df["source_url"] = url
    return df

for year in YEARS:
    for month in thai_months:
        url = f"{BASE_URL}ราคาทองย้อนหลัง-เดือน-{month}-{year}/"
        print(f"📆 กำลังดึงข้อมูล: {month} {year}")
        df = fetch_table(url)
        
        if df is not None and not df.empty:
            
            # ✅ ตั้งชื่อคอลัมน์ใหม่ตามชื่อที่กำหนดไว้ใน expected_headers
            df.rename(columns={
                'date_time': 'date',
                'gold_buy': 'gold_buy',         # ทองแท่ง รับซื้อ
                'gold_sell': 'gold_sell',       # ทองแท่ง ขายออก
                'gold_bar_buy': 'gold_bar_buy', # ทองรูปพรรณ ฐานภาษี
                'gold_bar_sell': 'gold_bar_sell'# ทองรูปพรรณ ขายออก
            }, inplace=True)
            
            # ✅ แยกวันที่และเวลา
            if 'date' in df.columns and df['date'] is not None:
                # แยกเวลา (เช่น 09:17)
                df['update_time'] = df['date'].astype(str).str.extract(r'(\d{1,2}:\d{2})') 
                # แยกวันที่ (เช่น 26/01/2566)
                df['date'] = df['date'].astype(str).str.extract(r'(\d{1,2}/\d{1,2}/\d{4})')
            else:
                df['update_time'] = None

            # ✅ ใส่ timestamp
            df['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            # ✅ เก็บเฉพาะคอลัมน์ที่ต้องการและทำความสะอาดข้อมูล
            keep_cols = [
                "date", "update_time", "gold_buy", "gold_sell",
                "gold_bar_buy", "gold_bar_sell", "timestamp", "source_url"
            ]
            
            for c in keep_cols:
                if c not in df.columns:
                    df[c] = None
                    
            df = df[keep_cols]
            
            # 🌟 ทำความสะอาดข้อมูล: ลบเครื่องหมายคอมม่าและแปลงเป็นตัวเลข
            for col in ["gold_buy", "gold_sell", "gold_bar_buy", "gold_bar_sell"]:
                if col in df.columns and df[col] is not None:
                    # ลบเครื่องหมายคอมม่าและช่องว่าง
                    df[col] = df[col].astype(str).str.replace(',', '', regex=False).str.replace(' ', '', regex=False)
                    # แปลงเป็นตัวเลข ถ้าไม่ใช่ตัวเลขให้เป็น NaN
                    df[col] = pd.to_numeric(df[col], errors='coerce') 
                
            # ✅ เก็บเฉพาะค่าล่าสุดของแต่ละวัน
            df = df.sort_values(by=["date", "update_time"], ascending=[True, True]).groupby("date", as_index=False).last()
            
            all_data.append(df)
            
driver.quit()

# ---------- SAVE ----------
if all_data:
    df_all = pd.concat(all_data, ignore_index=True)
    df_all = df_all.drop_duplicates(subset=["date"])
    os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)
    df_all.to_csv(SAVE_PATH, index=False, encoding="utf-8-sig", float_format="%.2f")
    print(f"✅ บันทึกข้อมูลทั้งหมด {len(df_all)} แถว ลงไฟล์: {SAVE_PATH}")
else:
    print("❌ ไม่พบข้อมูลในทุกเดือนที่ระบุ")

📆 กำลังดึงข้อมูล: ตุลาคม 2568
✅ บันทึกข้อมูลทั้งหมด 27 แถว ลงไฟล์: data/raw/gold_history_oct.csv


In [ ]:
# This file should run 2023-2025 because if you run only Oct 2025 the data will replace old data and missing
# If you want only Oct 2025 data please save in new file path.